In [1]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [3]:
raw_df = pd.read_csv('crypto_data.csv')

In [5]:
raw_df.sample(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
312,DUB,DubCoin,X15,True,PoW/PoS,NaN,0
471,LAZ,Lazarus,DPoS,True,DPoS,NaN,10000000
956,REDN,Reden,X16S,False,PoW,0.000000e+00,14000000
350,SOUL,SoulCoin,Scrypt,True,PoW/PoS,NaN,1400000000
690,ZENI,Zennies,Scrypt,True,PoS,NaN,1000000000
552,EBZ,Ebitz,PoS,False,PoS,NaN,21000000
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
524,SCT,ScryptToken,Scrypt,True,PoW/PoS,NaN,8842105
908,B2X,SegWit2x,X11,True,PoW/PoS,NaN,21000000
467,SMSR,Samsara Coin,QuBit,True,PoW/PoS,NaN,60000000
